In [2]:
from src.Tpm import *
from src.TpmTypes import *
mytpm = Tpm(useSimulator = True, host = '127.0.0.1', port = 2321)
mytpm.connect()

In [16]:
mytpm.close()

In [6]:
primaryHandle = TPM_HANDLE(TPM_RH.NULL)
# inSensitive = TPMS_SENSITIVE_CREATE(userAuth = b'This is my auth', data = b'This is my data')
# sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.AES, keyBits = 128, mode = TPM_ALG_ID.CFB)
sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.NULL, keyBits = None, mode = TPM_ALG_ID.NULL)
ecc = TPMS_ECC_PARMS(symmetric = sym, scheme = TPMS_SIG_SCHEME_ECDSA(hashAlg = TPM_ALG_ID.SHA256), curveID = TPM_ECC_CURVE.NIST_P256, kdf = TPMS_NULL_KDF_SCHEME())
# ecc = TPMS_ECC_PARMS(symmetric = sym, scheme = TPMS_SIG_SCHEME_ECDSA(hashAlg = None), curveID = TPM_ECC_CURVE.NIST_P256, kdf = TPM_ALG_ID.NULL)
# inPublic = TPMT_PUBLIC_PARMS(parameters = TPMS_SYMCIPHER_PARMS(sym = sym))
inPublic = TPMT_PUBLIC_PARMS(parameters = ecc)
outsideInfo = bytes.fromhex("")
creationPCR = TPMS_PCR_SELECTION(hash = TPM_ALG_ID.SHA256, pcrSelect = bytes.fromhex("00"))
ret1 = mytpm.CreatePrimary(primaryHandle, inSensitive, inPublic, outsideInfo, [])

800200000035000001314000000700000009400000090000010000000400000000000c002300100018000b00030010000000000000
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8002                 TPM_ST.SESSIONS
UINT32                                    |   .commandSize                               00000035             53
TPM_CC                                    |   .commandCode                               00000131             TPM_CC.CreatePrimary
TPMS_COMMAND_HANDLES_CREATE_PRIMARY       |   .handles                                                        
TPMI_RH_HIERARCHY                         |   |   .primaryHandle                         40000007             TPM_RH.NULL
UINT32                                    |   .authSize                                  00000009             9
TPMS_AUTH_COMMAND                         |   .authorizationArea[0]

AssertionError: Selection error in .parameters.inPublic.publicArea.parameters.eccDetail.symmetric.keyBits (TPMU_SYM_KEY_BITS): 0 not in {TPM_ALG.TDES: 'tdes', TPM_ALG.AES: 'aes', TPM_ALG.SM4: 'sm4', TPM_ALG.NULL: 'null', <class 'tpmstream.spec.structures.base_types.TPM_KEY_BITS'>: 'sym', TPM_ALG.XOR: 'xor'}

In [25]:
primaryHandle = TPM_HANDLE(TPM_RH.OWNER)
# inSensitive = TPMS_SENSITIVE_CREATE(userAuth = b'This is my auth', data = b'This is my data')
inSensitive = TPMS_SENSITIVE_CREATE(userAuth = bytes([]), data = bytes([]))
# sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.AES, keyBits = 128, mode = TPM_ALG_ID.CFB)
sym = TPMT_SYM_DEF_OBJECT(algorithm = TPM_ALG_ID.NULL, keyBits = 0, mode = TPM_ALG_ID.NULL) 
rsa = TPMS_RSA_PARMS(symmetric = sym, scheme = TPMS_ENC_SCHEME_OAEP(hashAlg=TPM_ALG_ID.SHA), keyBits = 2048, exponent = 65537)
# ecc = TPMS_ECC_PARMS(symmetric = sym, scheme = TPMS_SIG_SCHEME_ECDSA(hashAlg = None), curveID = TPM_ECC_CURVE.NIST_P256, kdf = TPM_ALG_ID.NULL)
# inPublic = TPMT_PUBLIC_PARMS(parameters = TPMS_SYMCIPHER_PARMS(sym = sym))
buffer = bytes(256)
inPublic = TPMT_PUBLIC(nameAlg = TPM_ALG_ID.SHA256, objectAttributes = TPMA_OBJECT.fixedTPM | TPMA_OBJECT.fixedParent | TPMA_OBJECT.userWithAuth | TPMA_OBJECT.sensitiveDataOrigin | TPMA_OBJECT.decrypt, authPolicy = None, parameters = rsa, unique = TPM2B_PUBLIC_KEY_RSA(buffer=None))
outsideInfo = bytes.fromhex("")
creationPCR = TPMS_PCR_SELECTION(hash = TPM_ALG_ID.SHA256, pcrSelect = bytes.fromhex("00"))
mytpm.allowErrors()
ret1 = mytpm.CreatePrimary(primaryHandle, inSensitive, inPublic, outsideInfo, [])

80020000004100000131400000010000000940000009000001000000040000000000180001000b0002007200000010001700040800000100010000000000000000
Command                                   .                                                                   
TPMI_ST_COMMAND_TAG                       |   .tag                                       8002                 TPM_ST.SESSIONS
UINT32                                    |   .commandSize                               00000041             65
TPM_CC                                    |   .commandCode                               00000131             TPM_CC.CreatePrimary
TPMS_COMMAND_HANDLES_CREATE_PRIMARY       |   .handles                                                        
TPMI_RH_HIERARCHY                         |   |   .primaryHandle                         40000001             TPM_RH.OWNER
UINT32                                    |   .authSize                                  00000009             9
TPMS_AUTH_COMMAND                         

TPMI_ALG_RSA_SCHEME                       |   |   |   |   |   |   |   .scheme            0017                 TPM_ALG.OAEP
TPMU_ASYM_SCHEME                          |   |   |   |   |   |   |   .details                                
TPMS_ENC_SCHEME_OAEP                      |   |   |   |   |   |   |   |   .oaep                               
TPMI_ALG_HASH                             |   |   |   |   |   |   |   |   |   .hashAlg   0004                 TPM_ALG.SHA
TPMI_RSA_KEY_BITS                         |   |   |   |   |   |   .keyBits               0800                 2048
UINT32                                    |   |   |   |   |   |   .exponent              00010001             65537
TPMU_PUBLIC_ID                            |   |   |   |   .unique                                             
TPM2B_PUBLIC_KEY_RSA                      |   |   |   |   |   .rsa                                            
UINT16                                    |   |   |   |   |   |   .size         